## kNN: Выбор метрики

In [1]:
import numpy as np

from sklearn.datasets import load_boston

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import scale

In [2]:
def write_answer(filename, answer):
    
    with open(filename, 'w') as fout:
        
        fout.write(str(answer))
        fout.close()

Главным параметром любого метрического алгоритма является функция расстояния (или метрика), используемая для измерения сходства между объектами. Можно использовать стандартный вариант (например, евклидову метрику), но гораздо более эффективным вариантом является подбор метрики под конкретную задачу. Один из подходов — использование той же евклидовой метрики, но с весами: каждой координате ставится в соответствие определенный коэффициент; чем он больше, тем выше вклад признака в итоговое расстояние. Веса настраиваются с целью оптимизации качества на отложенной выборке. Другой подход, о котором и пойдет речь в данном задании — выбор метрики из некоторого класса метрик. Мы возьмем за основу метрику Минковского:

Параметром метрики Минковского является число p, которое мы и будем настраивать.

$$\rho_p(x, z) = \left(\sum_{j=1}^d |x_j-y_j|^p\right)^{1/p}$$

### Реализация в sklearn

Нам понадобится решать задачу регрессии с помощью метода k ближайших соседей — воспользуйтесь для этого классом <code>sklearn.neighbors.KNeighborsRegressor</code>. Метрика задается с помощью параметра metric, нас будет интересовать значение 'minkowski'. Параметр метрики Минковского задается с помощью параметра p данного класса.

Мы будем использовать в данном задании набор данных Boston, где нужно предсказать стоимость жилья на основе различных характеристик расположения (загрязненность воздуха, близость к дорогам и т.д.). Подробнее о признаках можно почитать по адресу https://archive.ics.uci.edu/ml/machine-learning-databases/housing/

#### 1. Загрузите выборку Boston с помощью функции sklearn.datasets.load_boston(). Результатом вызова данной функции является объект, у которого признаки записаны в поле data, а целевой вектор — в поле target.

In [3]:
X, y = load_boston(return_X_y=True)

#### 2. Приведите признаки в выборке к одному масштабу при помощи функции sklearn.preprocessing.scale.

In [4]:
X_scaled = scale(X)

#### 3. Переберите разные варианты параметра метрики p по сетке от 1 до 10 с таким шагом, чтобы всего было протестировано 200 вариантов (используйте функцию numpy.linspace). Используйте KNeighborsRegressor с n_neighbors=5 и weights='distance' — данный параметр добавляет в алгоритм веса, зависящие от расстояния до ближайших соседей. В качестве метрики качества используйте среднеквадратичную ошибку (параметр scoring='mean_squared_error' у cross_val_score; при использовании библиотеки scikit-learn версии 0.18.1 и выше необходимо указывать scoring='neg_mean_squared_error').  Качество оценивайте, как и в предыдущем задании, с помощью кросс-валидации по 5 блокам с random_state = 42, не забудьте включить перемешивание выборки (shuffle=True).

In [5]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

max_cv_score, p_optim = 0, 0

for p in np.linspace(1, 10, 200):
    
    knn_reg = KNeighborsRegressor(n_neighbors=5, weights='distance', p=p, metric='minkowski')

    cv_score = np.abs(np.mean(cross_val_score(knn_reg, X_scaled, y, cv=kfold)))
    
    if cv_score > max_cv_score:
        max_cv_score = cv_score
        p_optim = p

#### 4. Определите, при каком p качество на кросс-валидации оказалось оптимальным. Обратите внимание, что cross_val_score возвращает массив показателей качества по блокам; необходимо максимизировать среднее этих показателей. Это значение параметра и будет ответом на задачу.

In [6]:
max_cv_score, p_optim

(0.8013825191275472, 1.0)

In [7]:
write_answer('submission_boston_1.txt', round(p_optim, 1))